In [68]:
import pandas as pd
from sqlalchemy import create_engine
import datetime
import numpy as np
from datetime import datetime as dt

### Store CSV into DataFrame

In [69]:
ebola_csv = "Resources/ebola_data.csv"
ebola_df = pd.read_csv(ebola_csv)

In [70]:
covid_csv = "Resources/covid_data.csv"
covid_df = pd.read_csv(covid_csv)

In [71]:
sars_csv = "Resources/sars_data.csv"
sars_df = pd.read_csv(sars_csv)

In [72]:
swineflu_csv = "Resources/swineflu.csv"
swineflu_df = pd.read_csv(swineflu_csv,encoding="ISO-8859-1")

# EBOLA DATA

In [111]:
#### Create new data with select columns
new_ebola_df = ebola_df[['id','Country', 'Date','No. of confirmed cases','No. of confirmed deaths']].copy()
new_ebola_df.head()

,id,Country,Date,No. of confirmed cases,No. of confirmed deaths
0,1,Guinea,8/29/2014,482.0,287.0
1,2,Nigeria,8/29/2014,15.0,6.0
2,3,Sierra Leone,8/29/2014,935.0,380.0
3,4,Liberia,8/29/2014,322.0,225.0
4,5,Sierra Leone,9/5/2014,1146.0,443.0


In [113]:
### Clean DataFrame
ebola_virus=new_ebola_df.rename(columns={"Date": "date","No. of confirmed cases": "cases_confirmed", "No. of confirmed deaths": "deaths_confirmed","Country": "country"})

ebola_virus.style.format({
    'cases_confirmed': '{:,.0f}'.format,
    'deaths_confirmed': '{:,.0f}'.format,
})
ebola_virus.head()

,id,country,date,cases_confirmed,deaths_confirmed
0,1,Guinea,8/29/2014,482.0,287.0
1,2,Nigeria,8/29/2014,15.0,6.0
2,3,Sierra Leone,8/29/2014,935.0,380.0
3,4,Liberia,8/29/2014,322.0,225.0
4,5,Sierra Leone,9/5/2014,1146.0,443.0


# CORONA DATA



In [86]:
# Preview dataframe
covid_df.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [87]:
# Define needed columns
col = ['SNo', 'Country/Region', 'ObservationDate', 'Deaths']

# Create copy of dataframe using columns above
new_covid_df = covid_df[col].copy()

# Normalize column names of dataframe and preview
new_covid_df = new_covid_df.rename(columns={'SNo':'id', 'Country/Region':'country', 'ObservationDate':'date', 'Deaths':'deaths'})
new_covid_df.head()

,id,country,date,deaths
0,1,Mainland China,01/22/2020,0.0
1,2,Mainland China,01/22/2020,0.0
2,3,Mainland China,01/22/2020,0.0
3,4,Mainland China,01/22/2020,0.0
4,5,Mainland China,01/22/2020,0.0


In [88]:
# Review stats of dataframe
new_covid_df.describe()

,id,deaths
count,3395.000000,3395.000000
mean,1698.000000,17.756112
std,980.196409,187.195366
min,1.000000,0.000000
25%,849.500000,0.000000
50%,1698.000000,0.000000
75%,2546.500000,1.000000
max,3395.000000,2871.000000


In [89]:
# Group by date and country
new_covid_df = new_covid_df.groupby(['date', 'country'])

In [90]:
# Sum deaths on group by object and preview
deaths = new_covid_df['deaths'].sum()
deaths.head()

date        country       
01/22/2020  Hong Kong          0.0
            Japan              0.0
            Macau              0.0
            Mainland China    17.0
            South Korea        0.0
Name: deaths, dtype: float64

In [92]:
# Transformed data ready to be loaded to db
new_covid_df.head()

,id,country,date,deaths
0,1,Mainland China,01/22/2020,0.0
1,2,Mainland China,01/22/2020,0.0
2,3,Mainland China,01/22/2020,0.0
3,4,Mainland China,01/22/2020,0.0
4,5,Mainland China,01/22/2020,0.0
...,...,...,...,...
3372,3373,Poland,03/04/2020,0.0
3373,3374,Saudi Arabia,03/04/2020,0.0
3374,3375,Sri Lanka,03/04/2020,0.0
3375,3376,Tunisia,03/04/2020,0.0


In [101]:
# Transformed data ready to be loaded to db
new_covid_df.head()

,id,country,date,deaths
0,1,Mainland China,01/22/2020,0.0
1,2,Mainland China,01/22/2020,0.0
2,3,Mainland China,01/22/2020,0.0
3,4,Mainland China,01/22/2020,0.0
4,5,Mainland China,01/22/2020,0.0
...,...,...,...,...
3372,3373,Poland,03/04/2020,0.0
3373,3374,Saudi Arabia,03/04/2020,0.0
3374,3375,Sri Lanka,03/04/2020,0.0
3375,3376,Tunisia,03/04/2020,0.0


# SARS DATA

In [16]:
#View dataset
sars_df.head()


,Date,Country,Cumulative number of case(s),Number of deaths,Number recovered
0,2003-03-17,Germany,1,0,0
1,2003-03-17,Canada,8,2,0
2,2003-03-17,Singapore,20,0,0
3,2003-03-17,"Hong Kong SAR, China",95,1,0
4,2003-03-17,Switzerland,2,0,0


In [17]:
#Select columns for ETL transfer
new_sars_df = sars_df[['Country', 'Date', 'Cumulative number of case(s)', 'Number of deaths']].copy()
new_sars_df.head()

,Country,Date,Cumulative number of case(s),Number of deaths
0,Germany,2003-03-17,1,0
1,Canada,2003-03-17,8,2
2,Singapore,2003-03-17,20,0
3,"Hong Kong SAR, China",2003-03-17,95,1
4,Switzerland,2003-03-17,2,0


In [18]:
#Change column names
sars_virus = new_sars_df.rename(columns={'Country':'country', 'Date':'date', 'Cumulative number of case(s)':'cases_confirmed', 'Number of deaths':'deaths_confirmed'})
sars_virus

,country,date,cases_confirmed,deaths_confirmed
0,Germany,2003-03-17,1,0
1,Canada,2003-03-17,8,2
2,Singapore,2003-03-17,20,0
3,"Hong Kong SAR, China",2003-03-17,95,1
4,Switzerland,2003-03-17,2,0
...,...,...,...,...
2533,Switzerland,2003-07-11,1,0
2534,Thailand,2003-07-11,9,2
2535,United Kingdom,2003-07-11,4,0
2536,United States,2003-07-11,75,0


In [19]:
    #Review data
sars_virus.describe()

,cases_confirmed,deaths_confirmed
count,2538.000000,2538.000000
mean,219.211978,17.812845
std,825.024564,61.425627
min,0.000000,0.000000
25%,1.000000,0.000000
50%,3.000000,0.000000
75%,14.750000,2.000000
max,5329.000000,348.000000


In [20]:
#View data for revelent information
#sars_deaths = sars_virus.groupby(['date', 'country'])

In [21]:
#group data to verify clean
total_deaths = sars_deaths['deaths_confirmed'].sum()
total_deaths

date        country             
2003-03-17  Canada                   2
            Germany                  0
            Hong Kong SAR, China     1
            Singapore                0
            Switzerland              0
                                    ..
2003-07-11  Taiwan, China           84
            Thailand                 2
            United Kingdom           0
            United States            0
            Viet Nam                 5
Name: deaths_confirmed, Length: 2537, dtype: int64

In [22]:
#View clean data
total_deaths_df = pd.DataFrame(total_deaths)
total_deaths_df.head()

deaths_confirmed
date       country                               
2003-03-17 Canada                               2
           Germany                              0
           Hong Kong SAR, China                 1
           Singapore                            0
           Switzerland                          0

In [23]:
#FINAL DATA SET
sars_virus.head()

,country,date,cases_confirmed,deaths_confirmed
0,Germany,2003-03-17,1,0
1,Canada,2003-03-17,8,2
2,Singapore,2003-03-17,20,0
3,"Hong Kong SAR, China",2003-03-17,95,1
4,Switzerland,2003-03-17,2,0


# SWINE FLU DATA

In [24]:
swineflu_df.describe()

,id,Cases,Deaths
count,1822.000000,1822.000000,1821.000000
mean,911.500000,901.063666,4.315761
std,526.110413,5264.291392,25.793340
min,1.000000,1.000000,0.000000
25%,456.250000,3.000000,0.000000
50%,911.500000,13.500000,0.000000
75%,1366.750000,88.750000,0.000000
max,1822.000000,94512.000000,429.000000


In [25]:
swineflu_df


,id,Country,Cases,Deaths,Update Time
0,1,Algeria,5,0.0,7/6/2009 9:00
1,2,Antigua and Barbuda,2,0.0,7/6/2009 9:00
2,3,Argentina,2485,60.0,7/6/2009 9:00
3,4,Australia,5298,10.0,7/6/2009 9:00
4,5,Austria,19,0.0,7/6/2009 9:00
...,...,...,...,...,...
1817,1818,Thailand,2,0.0,5/23/2009 8:00
1818,1819,Turkey,2,0.0,5/23/2009 8:00
1819,1820,United Kingdom,117,0.0,5/23/2009 8:00
1820,1821,United States of America,6552,9.0,5/23/2009 8:00


In [26]:
swineflu_df=swineflu_df.rename(columns={"Update Time": "date","Cases": "cases_confirmed", "Deaths": "deaths_confirmed","Country": "country"})

In [27]:
swineflu_df

,id,country,cases_confirmed,deaths_confirmed,date
0,1,Algeria,5,0.0,7/6/2009 9:00
1,2,Antigua and Barbuda,2,0.0,7/6/2009 9:00
2,3,Argentina,2485,60.0,7/6/2009 9:00
3,4,Australia,5298,10.0,7/6/2009 9:00
4,5,Austria,19,0.0,7/6/2009 9:00
...,...,...,...,...,...
1817,1818,Thailand,2,0.0,5/23/2009 8:00
1818,1819,Turkey,2,0.0,5/23/2009 8:00
1819,1820,United Kingdom,117,0.0,5/23/2009 8:00
1820,1821,United States of America,6552,9.0,5/23/2009 8:00


In [28]:
swineflu_df.dtypes

id                    int64
country              object
cases_confirmed       int64
deaths_confirmed    float64
date                 object
dtype: object

In [29]:
print(swineflu_df['date'])

0        7/6/2009 9:00
1        7/6/2009 9:00
2        7/6/2009 9:00
3        7/6/2009 9:00
4        7/6/2009 9:00
             ...      
1817    5/23/2009 8:00
1818    5/23/2009 8:00
1819    5/23/2009 8:00
1820    5/23/2009 8:00
1821    5/23/2009 8:00
Name: date, Length: 1822, dtype: object


In [30]:
#swineflu_df['date'] = swineflu_df['date'].dt.normalize()
swineflu_df['date'] = pd.to_datetime(swineflu_df['date'], errors='coerce').dt.date

In [31]:
swineflu_df

,id,country,cases_confirmed,deaths_confirmed,date
0,1,Algeria,5,0.0,2009-07-06
1,2,Antigua and Barbuda,2,0.0,2009-07-06
2,3,Argentina,2485,60.0,2009-07-06
3,4,Australia,5298,10.0,2009-07-06
4,5,Austria,19,0.0,2009-07-06
...,...,...,...,...,...
1817,1818,Thailand,2,0.0,2009-05-23
1818,1819,Turkey,2,0.0,2009-05-23
1819,1820,United Kingdom,117,0.0,2009-05-23
1820,1821,United States of America,6552,9.0,2009-05-23


In [32]:
swineflu_df.count()

id                  1822
country             1822
cases_confirmed     1822
deaths_confirmed    1821
date                1822
dtype: int64

In [33]:
swineflu_virus = swineflu_df
swineflu_virus.head()

,id,country,cases_confirmed,deaths_confirmed,date
0,1,Algeria,5,0.0,2009-07-06
1,2,Antigua and Barbuda,2,0.0,2009-07-06
2,3,Argentina,2485,60.0,2009-07-06
3,4,Australia,5298,10.0,2009-07-06
4,5,Austria,19,0.0,2009-07-06


### Connect to local database

In [34]:
rds_connection_string = "postgres:postgres@localhost:5432/virus_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [36]:
engine.table_names()

['ebola_virus', 'corona_virus', 'sars_virus', 'swineflu_virus']

### Use pandas to load csv converted DataFrame into database

In [37]:
ebola_virus.to_sql(name='country', con=engine, if_exists='append', index=False)

In [39]:
sars_virus.to_sql(name='country', con=engine, if_exists='append', index=False)

In [40]:
swineflu_virus.to_sql(name='country', con=engine, if_exists='append', index=False)

In [83]:
corona_virus.to_sql(name='country', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "deaths" of relation "country" does not exist
LINE 1: INSERT INTO country (id, country, date, deaths) VALUES (1, '...
                                                ^

[SQL: INSERT INTO country (id, country, date, deaths) VALUES (%(id)s, %(country)s, %(date)s, %(deaths)s)]
[parameters: ({'id': 1, 'country': 'Mainland China', 'date': '01/22/2020', 'deaths': 0.0}, {'id': 2, 'country': 'Mainland China', 'date': '01/22/2020', 'deaths': 0.0}, {'id': 3, 'country': 'Mainland China', 'date': '01/22/2020', 'deaths': 0.0}, {'id': 4, 'country': 'Mainland China', 'date': '01/22/2020', 'deaths': 0.0}, {'id': 5, 'country': 'Mainland China', 'date': '01/22/2020', 'deaths': 0.0}, {'id': 6, 'country': 'Mainland China', 'date': '01/22/2020', 'deaths': 0.0}, {'id': 7, 'country': 'Mainland China', 'date': '01/22/2020', 'deaths': 0.0}, {'id': 8, 'country': 'Mainland China', 'date': '01/22/2020', 'deaths': 0.0}  ... displaying 10 of 3395 total boun

### Use pandas to load json converted DataFrame into database

In [104]:
ebola_virus.to_sql(name='ebola_virus', con=engine, if_exists='append', index=False)

In [106]:
sars_virus.to_sql(name='sars_virus', con=engine, if_exists='append', index=False)

In [107]:
swineflu_virus.to_sql(name='swineflu_virus', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the ebola_virus table

In [108]:
#TABLE - EBOLA VIRUS

pd.read_sql_query('select * from ebola_virus', con=engine).head()

,id,country,date,cases_confirmed,deaths_confirmed
0,1,Guinea,2014-08-29,482.0,287.0
1,2,Nigeria,2014-08-29,15.0,6.0
2,3,Sierra Leone,2014-08-29,935.0,380.0
3,4,Liberia,2014-08-29,322.0,225.0
4,5,Sierra Leone,2014-09-05,1146.0,443.0


In [109]:
#TABLE - SARS VIRUS
pd.read_sql_query('select * from sars_virus', con=engine).head()

,id,country,date,cases_confirmed,deaths_confirmed
0,1,Germany,2003-03-17,1,0
1,2,Canada,2003-03-17,8,2
2,3,Singapore,2003-03-17,20,0
3,4,"Hong Kong SAR, China",2003-03-17,95,1
4,5,Switzerland,2003-03-17,2,0


In [110]:
#TABLE - SWINE FLU VIRUS
pd.read_sql_query('select * from swineflu_virus', con=engine).head()

,id,country,cases_confirmed,deaths_confirmed,date
0,1,Algeria,5,0.0,2009-07-06
1,2,Antigua and Barbuda,2,0.0,2009-07-06
2,3,Argentina,2485,60.0,2009-07-06
3,4,Australia,5298,10.0,2009-07-06
4,5,Austria,19,0.0,2009-07-06
